# 一、蒙特卡罗方法

## 1. MCMC（马尔科夫链蒙特卡罗Markov Chain Monte Carlo）
+ 从名字我们可以看出，MCMC由两个MC组成，即蒙特卡罗方法（Monte Carlo Simulation，简称MC）和马尔科夫链（Markov Chain ，也简称MC）。要弄懂MCMC的原理我们首先得搞清楚蒙特卡罗方法和马尔科夫链的原理。我们将用三篇来完整学习MCMC。在本篇，我们关注于蒙特卡罗方法。

## 2. 蒙特卡罗方法引入

+ 蒙特卡罗原来是一个赌场的名称，用它作为名字大概是因为蒙特卡罗方法是一种随机模拟的方法，这很像赌博场里面的扔骰子的过程。最早的蒙特卡罗方法都是为了求解一些不太好求解的求和或者积分问题。比如积分：
 $$\theta = \int_{a}^{b} f(x) dx \tag{1}$$

+ 如果$f(x)$的原函数很难求解或不存在，就可以借助蒙特卡罗方法模拟近似求解，如图：
<div align="center"> <img src="media/mc1.png" width="50%" /> </div>

+ 一个简单的近似求解的方法：在[a, b]之间随机采样一个点，例如$x_0$，然后用$f(x_0)$代表所有$f(x)$的值，那定积分的近似解为：
$$(b - a)f(x_0) \tag{2}$$

+ 可以发现：这样一个值代表所有$f(x)$值太粗糙，那么我们可以在[a, b]区间采集n个值：$x_0,x_1,x_2,\cdots,x_{n-1}$用它们的均值来代表$f(x)$的值，此时定积分的近似解为：
$$(b-a)\frac{1}{n}\sum_{0}^{n-1}f(x_i) \tag{3}$$

### 2.1 重要性采样法

+ 上面的结论隐含一个前提，x在[a, b]之间是均匀分布的，而实际上绝大部分情况不是均匀分布，所以上面的方法结果可能与真实值相差甚远；
+ 可以想象，如果我们知道x在[a, b]之间的概率分布函数$p(x)$，那么定积分就可以近似：
$$\theta = \int_{a}^{b}f(x)dx = \int_{a}^{b}\frac{f(x)}{p(x)}p(x)dx =E[\frac{f(x)}{p(x)}] \approx \frac{1}{n}\sum_{0}^{n-1}\frac{f(x_i)}{p(x_i)} \tag{4}$$


+ 上式中，$\frac{1}{p(x_i)}$相当于一个由概率的倒数构造的权重，概率越大出现的几率越大，赋予一个较小的权重，相当于根据重要性进行采样；

+ 可以看出，$p(x)$为均匀分布时，$p(x_i)=1/(b-a)$，为上面的特例；
$$\frac{1}{n}\sum_{0}^{n-1}\frac{f(x_i)}{p(x_i)}=\frac{1}{n}\sum_{0}^{n-1}\frac{f(x_i)}{1/(b-a)}=(b-a)\frac{1}{n}\sum_{0}^{n-1}f(x_i) \tag{5}$$

+ 现在问题转到了如何找到$x$的分布$p(x)$对应的若干个样本上来。

+ 重要性采样法求定积分$\int_2^5x^2dx=39$的R语言代码实例:

In [11]:
a <- 2
b <- 5
f <- function(x) {
  x^2
}
s_f <- function(N) {
  x <- runif(N, a, b)
  return((b - a) * mean(f(x)))
}
s_f(10000)

[1] 39.08295

### 2.2 投点法

+ 向一个区域内投点，根据落在不同区域的概率等于区域的面积比来计算定积分；

+ 原理：
$$ \frac{s_0}{s}=\frac{\int_a^bf(x)dx}{s_{矩形}}=\frac{n}{N}=p \tag{6}$$
$$\int_a^bf(x)dx=s_{矩形}*\frac{n}{N} \tag{7}$$

+ R代码示例：

In [13]:
a <- 2
b <- 5
f <- function(x) {
  x^2
}
s_f <- function(N) {
  x <- runif(N, a, b)
  d <- max(f(x))
  y <- runif(N, 0, d)
  n <- length(x[which(y < f(x))])
  return((b - a) * d * (n / N))
}
s_f(10000)

[1] 39.29207

## 3. 概率分布采样
+ 蒙特卡罗方法的关键是得到$x$的概率分布。如果求出了$x$的概率分布，我们可以基于概率分布去采样基于这个概率分布的n个$x$的样本集，带入蒙特卡罗求和的式子即可求解。但是还有一个关键的问题需要解决，即如何基于概率分布去采样基于这个概率分布的n个$x$的样本集。
+ 对于常见的均匀分布𝑢𝑛𝑖𝑓𝑜𝑟𝑚(0,1)是非常容易采样样本的，一般通过线性同余发生器可以很方便的生成(0,1)之间的伪随机数样本。而其他常见的概率分布，无论是离散的分布还是连续的分布，它们的样本都可以通过𝑢𝑛𝑖𝑓𝑜𝑟𝑚(0,1)的样本转换而得。比如二维正态分布的样本$(𝑍_1,𝑍_2)$可以通过通过独立采样得到的𝑢𝑛𝑖𝑓𝑜𝑟𝑚(0,1)样本对$(𝑋_1,𝑋_2)$通过如下的式子转换而得：
+ 其他一些常见的连续分布，比如t分布，F分布，Beta分布，Gamma分布等，都可以通过类似的方式从𝑢𝑛𝑖𝑓𝑜𝑟𝑚(0,1)得到的采样样本转化得到。在python的numpy，scikit-learn等类库中，都有生成这些常用分布样本的函数可以使用。
+ 不过很多时候，我们的𝑥的概率分布不是常见的分布，这意味着我们没法方便的得到这些非常见的概率分布的样本集。那这个问题怎么解决呢？

## 4. 接受拒绝采样
+ 对于概率分布不是常见的分布，一个可行的办法是采用接受-拒绝采样来得到该分布的样本。既然 𝑝(𝑥) 太复杂在程序中没法直接采样，那么我设定一个程序可采样的分布 𝑞(𝑥) 比如高斯分布，然后按照一定的方法拒绝某些样本，以达到接近 𝑝(𝑥) 分布的目的，其中𝑞(𝑥)叫做 proposal distribution。
<div align="center"> <img src="media/mc2.png" width="50%" /> </div>

+ 具体采用过程如下，设定一个方便采样的常用概率分布函数 𝑞(𝑥)，以及一个常量 𝑘，使得 𝑝(𝑥) 总在 𝑘𝑞(𝑥) 的下方。如上图。

+ 首先，采样得到𝑞(𝑥)的一个样本$z_0$，采样方法如第三节。然后，从均匀分布$(0,𝑘𝑞(𝑧_0))$中采样得到一个值𝑢。如果𝑢落在了上图中的灰色区域，则拒绝这次抽样，否则接受这个样本$z_0$。重复以上过程得到n个接受的样本$z_0,z_1,z_2,\cdots,z_{n-1}$，则最后的蒙特卡罗方法求解结果为：
$$\frac{1}{n}\sum_{i=0}^{n-1}\frac{f(z_i)}{p(z_i)}$$

+ 整个过程中，我们通过一系列的接受拒绝决策来达到用𝑞(𝑥)模拟𝑝(𝑥)概率分布的目的。

## 5. 蒙特卡罗方法小结
+ 使用接受-拒绝采样，我们可以解决一些概率分布不是常见的分布的时候，得到其采样集并用蒙特卡罗方法求和的目的。但是接受-拒绝采样也只能部分满足我们的需求，在很多时候我们还是很难得到我们的概率分布的样本集。比如：
  + 对于一些二维分布$𝑝(𝑥,𝑦)$，有时候我们只能得到条件分布$𝑝(𝑥|𝑦)$和$𝑝(𝑦|𝑥)$和,却很难得到二维分布$𝑝(𝑥,𝑦)$一般形式，这时我们无法用接受-拒绝采样得到其样本集。
  + 对于一些高维的复杂非常见分布$𝑝(𝑥_1,𝑥_2,...,𝑥_𝑛)$，我们要找到一个合适的$𝑞(𝑥)$和$𝑘$非常困难。
+ 从上面可以看出，要想将蒙特卡罗方法作为一个通用的采样模拟求和的方法，必须解决如何方便得到各种复杂概率分布的对应的采样样本集的问题。而我们下一篇要讲到的马尔科夫链就是帮助找到这些复杂概率分布的对应的采样样本集的白衣骑士。下一篇我们来总结马尔科夫链的原理。